In [4]:
#just run this
from google.colab import drive
import os
import json
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
import matplotlib.pyplot as plt
from keras.datasets import mnist
import tensorflow as tf


import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image
from keras.datasets import mnist
from PIL import Image as pil_image


def inception_score(x, resizer=None, batch_size=32, denorm_im=1):
    r = None
    n_batch = (x.shape[0]+batch_size-1) // batch_size
    for j in range(n_batch):
        x_batch = x[j*batch_size:(j+1)*batch_size, :, :, :]
        if denorm_im:
          x_batch = (x_batch + 1)/2
        r_batch = inception_score_model.predict(x_batch) # r has the probabilities for all classes
        r = r_batch if r is None else np.concatenate([r, r_batch], axis=0)
    p_y = np.mean(r, axis=0) # p(y)
    e = r*np.log(r/p_y) # p(y|x)log(P(y|x)/P(y))
    e = np.sum(e, axis=1) # KL(x) = Σ_y p(y|x)log(P(y|x)/P(y))
    e = np.mean(e, axis=0)
    return np.exp(e) # Inception score


def image_inception_score(generator, n_ex=10000, dim_random=10, input_noise=None, denorm_im=1):
    if input_noise is None:
      input_noise = np.random.normal(0,1,size=[n_ex,dim_random])
    x_pred = generator.predict(input_noise)
    if len(x_pred.shape)==2:
      x_pred = x_pred.reshape(n_ex, 28, 28, 1)
    return inception_score(x_pred, denorm_im=denorm_im)


def plot_history(history, metric = None):
  # Plots the loss history of training and validation (if existing)
  # and a given metric
  # Be careful because the axis ranges are automatically adapted
  # which may not desirable to compare different runs.
  # Also, in some cases you may want to combine several curves in one
  # figure for easier comparison, which this function does not do.
  
  if metric != None:
    fig, axes = plt.subplots(2,1)
    axes[0].plot(history.history[metric])
    try:
      axes[0].plot(history.history['val_'+metric])
      axes[0].legend(['Train', 'Val'])
    except:
      pass
    axes[0].set_title('{:s}'.format(metric))
    axes[0].set_ylabel('{:s}'.format(metric))
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    axes[1].plot(history.history['loss'])
    try:
      axes[1].plot(history.history['val_loss'])
      axes[1].legend(['Train', 'Val'])
    except:
      pass
    axes[1].set_title('Model Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
  else:
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')

def train_epoch(gan, generator, discriminator, plt_frq=25, BATCH_SIZE=32, mnist=1):  
  vector_ind = np.random.permutation(x_train.shape[0])
  nb_epoch = int(x_train.shape[0]/BATCH_SIZE)
  pbar = tqdm_notebook(range(nb_epoch))
  for e in range(nb_epoch):  
    ind = vector_ind[e*BATCH_SIZE:(e+1)*BATCH_SIZE]
    # Make generative images
    image_batch = x_train[ind,:,:,:]    
    noise_gen = np.random.normal(0,1,size=[BATCH_SIZE,randomDim])
    generated_images = generator.predict(noise_gen)
    # Train discriminator on generated images
    X = np.concatenate((image_batch, generated_images))
    y = np.zeros([2*BATCH_SIZE])
    y[0:BATCH_SIZE] = 1
    y[BATCH_SIZE:] = 0

    #make_trainable(discriminator,True)
    d_loss  = discriminator.train_on_batch(X,y)
    losses["d"].append(d_loss)
    # train Generator-Discriminator stack on input noise to non-generated output class
    noise_tr = np.random.normal(0,1,size=[BATCH_SIZE,randomDim])
    y2 = np.zeros([BATCH_SIZE])
    y2[:] = 1

    #make_trainable(discriminator,False)
    g_loss = gan.train_on_batch(noise_tr, y2 )
    losses["g"].append(g_loss)

    # Updates plots. This is a little bit of a mess due to how the notebook
    # handles the outputs
    if e % plt_frq==plt_frq-1:
      plot_loss(losses)
      plot_gen(mnist)
      fig, ax = plt.subplots(2,1, figsize=(20,10) )
      img=mpimg.imread('loss.png')
      ax[0].imshow(img)
      ax[0].axis('off')
      img=mpimg.imread('images.png')
      ax[1].imshow(img)
      ax[1].axis('off')
      plt.tight_layout()
      display.clear_output(wait=True)
      pbar.update(plt_frq)
      display.display(pbar)
      display.display(fig)
      plt.close()
      
def plot_loss(losses):
    plt.figure()
    plt.plot(losses["d"], label='discriminitive loss')
    plt.plot(losses["g"], label='generative loss')
    plt.legend()
    plt.savefig('./loss.png')
    plt.close()
def plot_gen(mnist=1, n_ex=16, dim=(4,4), figsize=(10,10)):
    noise = np.random.normal(0,1,size=[n_ex,randomDim])
    generated_images = generator.predict(noise)
    
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        if mnist:
          img = generated_images[i,:,:,0]
          plt.imshow(img, cmap='gray')
        else:
          img = generated_images[i,:,:,:]
          plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./images.png')
    plt.close()

  

In [ ]:
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Deep Learning 2021/Notebook Seven - VAE GAN")
!ls

Mounted at /content/drive
 07_VAE_GAN.ipynb  'Copy of Copy of 07_VAE_GAN.ipynb'  'Task One'


In [5]:
import numpy as np
import keras
np.random.seed(123)  # for reproducibility
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Layer, Input, Lambda 
from keras.layers import Multiply, Add, BatchNormalization, Reshape
from keras.layers import UpSampling2D, Convolution2D, LeakyReLU, Flatten, ReLU


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from keras.datasets import mnist
from keras import backend as K
from scipy.stats import norm

import matplotlib.image as mpimg
import sys

from tqdm import tqdm_notebook
from IPython import display
%matplotlib inline

from keras import initializers
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D

In [6]:
from keras.layers import Dropout, merge, concatenate, UpSampling2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Reshape, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D, Input, Dense, Reshape, Flatten
from keras.optimizers import Adam

def build_generator(im_shape):
  
  img_B = Input(shape=(im_shape[0], im_shape[1], 1))
  
  ## Encoder part
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(img_B)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  drop4 = Dropout(rate=0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  drop5 = Dropout(rate=0.5)(conv5)

  ## Now the decoder starts
  up6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)

  up7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)

  up8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)

  up9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv10 = Conv2D(3, 3,  padding = 'same')(conv9)

  model = Model(inputs = img_B, outputs = conv10, name='generator')
  
  return model

In [7]:
def build_discriminator(im_shape):
  
  img_A = Input(shape=(im_shape[0], im_shape[1], 3))
  img_B = Input(shape=(im_shape[0], im_shape[1], 1))

  combined_imgs = concatenate([img_A, img_B], axis=-1)
    
  disc_layer = Conv2D(64, kernel_size=(5, 5), padding='same', strides=(2, 2))(combined_imgs)
  disc_layer = LeakyReLU(alpha=0.2)(disc_layer)
  
  disc_layer = Conv2D(64, kernel_size=(5, 5), padding='same', strides=(2, 2))(disc_layer)
  disc_layer = BatchNormalization(momentum=0.8)(disc_layer)
  disc_layer = LeakyReLU(alpha=0.2)(disc_layer)

  disc_layer = Conv2D(128, kernel_size=(5, 5), strides=(2, 2))(disc_layer)
  disc_layer = BatchNormalization(momentum=0.8)(disc_layer)
  disc_layer = LeakyReLU(alpha=0.2)(disc_layer)
  
  disc_layer = Flatten()(disc_layer)
  disc_layer = Dense(1024)(disc_layer)

  prob = Dense(1, name="disc_dense")(disc_layer)

  discriminator = Model(inputs=[img_A, img_B], outputs=[prob], name='discriminator')
  
  return discriminator

In [ ]:
#DATA

original_dim = 784
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, original_dim) / 255.
x_test = x_test.reshape(-1, original_dim) / 255. 

#INCEPTION SCORE MODEL

!wget https://imperialcollegelondon.box.com/shared/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5 -O inception_score_mnist.h5
inception_score_model = keras.models.load_model('./inception_score_mnist.h5')

11493376/11490434 [==============================] - 0s 0us/step
--2021-03-29 10:43:31--  https://imperialcollegelondon.box.com/shared/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 103.116.4.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5 [following]
--2021-03-29 10:43:31--  https://imperialcollegelondon.box.com/public/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5 [following]
--2021-03-29 10:43:31--  https://imperialcollegelondon.app.box.com/public/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5
Resolving imperia

# Coursework


## Task 2: Quantitative VS Qualitative Results

In this task, we will observe the difference between two trained models for colouring images. One is the model trained during the tutorial, which uses a cGAN approach to predict the RGB pixel-wise values of a B&W image. The other one is a simple UNet autoencoder trained with a Mean Absolute Error (MAE) loss, which is trained to predict directly the RBG image without any GAN based learning strategy. We refer to the first and second models as cGAN and MAE models, respectively. For this task, 20 epochs trained weights for the cGAN and MAE models are provided. If desired, the code to train the MAE model can be found below:

In [ ]:
# Input images and their conditioning images

# Load the data, shuffled and split between train and test sets
dataset_loader = DataLoader(dataset_name = 'CIFAR10')

training_shape = dataset_loader.get_dataset_shape()
test_shape = dataset_loader.get_dataset_shape(is_training=False)

print('Shape of Training Images: {}'.format(training_shape))
print('Shape of Test Images: {}'.format(test_shape))
im_shape = (32, 32)
img_A = Input(shape=(im_shape[0], im_shape[1], 3))
img_B = Input(shape=(im_shape[0], im_shape[1], 1))

# Build the generator
generator_mae = build_generator(im_shape)
fake_A = generator_mae(img_B)
generator_mae = Model(inputs=img_B, outputs=fake_A)
generator_mae.compile(optimizer = 'Adam', loss = 'mean_absolute_error', metrics = ['mae'])

num_epochs = 1
batch_size = 128
n_batches = dataset_loader.get_num_batches(batch_size)
     
for epoch in range(num_epochs):
  
  start_time = datetime.datetime.now()
  
  # Record average losses. Monitorize the loss function. 
  g_avg_loss = []

  # load_batch() returns a batch generator
  # Before starting the epoch, it shuffles the dataset
  for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(batch_size)):
    
    # -----------------
    #  Train Generator (MAE)
    # -----------------
    g_loss = generator_mae.train_on_batch(imgs_B, imgs_A)
    g_avg_loss.append(g_loss[0])

    fake_A = generator_mae.predict(imgs_B)  

    elapsed_time = datetime.datetime.now() - start_time
    
    # Aproximation of epoch remaining time
    remaining_time = (elapsed_time/(batch_i+1)) * (n_batches-batch_i-1)

    # Plot examples          
    if batch_i%50 == 0:
      showColoredIms(imgs_B, fake_A, imgs_A)      
    
    # Plot the progress
    if batch_i%10 == 0:          

      print ("[Epoch %d/%d] [Batch %d/%d] [G loss: %f] elapsed_time: %s  remaining_time: %s" % (epoch, num_epochs,
              batch_i, n_batches, np.mean(g_avg_loss), elapsed_time, remaining_time))            
        
  # Saves optimizer and weights
  generator_mae.save('generator_mae.h5') 

NameError: ignored

Instead of training the models, we can directly load their pre-trained weights by running:

In [8]:
!wget -O weights.zip https://imperialcollegelondon.box.com/shared/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
!unzip -q ./weights.zip
!rm ./weights.zip

!wget -O weights_mae.zip https://imperialcollegelondon.box.com/shared/static/empxfp2v05xp7h8rwsccuiy35c3vddzd.zip
!unzip -q ./weights_mae.zip
!rm ./weights_mae.zip

generator_cGAN = keras.models.load_model('./weights/generator.h5')
generator_mae = keras.models.load_model('mae_generator.h5', compile=False)

generator_cGAN.compile(optimizer='Adam', loss='mean_absolute_error', metrics=['mae'])
generator_mae.compile(optimizer='Adam', loss='mean_absolute_error', metrics=['mae'])

--2021-03-29 18:58:34--  https://imperialcollegelondon.box.com/shared/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 185.235.236.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip [following]
--2021-03-29 18:58:34--  https://imperialcollegelondon.box.com/public/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip [following]
--2021-03-29 18:58:34--  https://imperialcollegelondon.app.box.com/public/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
Resolving imperialcollegelondon.app.box.com (imperialcollegelondon.app.bo

We have loaded both models, and we are ready to compare them. In this task, you are asked to analyse the difference between the quantitative versus the qualitative results. To do so, we provided two pieces of code. The first one will compute the MAE metric for both models in the test dataset. As we know, this metric is widely used on image generation tasks, such as image upsampling, image reconstruction, image translation, and so on.

In [9]:
g_mae_avg_mae = []
for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(128, is_training=False)):
    _, mae = generator_mae.evaluate(imgs_B, imgs_A, verbose=0)
    g_mae_avg_mae.append(mae)

print("MAE (Trained MAE): {:.4f}".format(np.mean(g_mae_avg_mae)))

g_cgan_avg_mae = []
for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(128, is_training=False)):
    _, mae = generator_cGAN.evaluate(imgs_B, imgs_A, verbose=0)
    g_cgan_avg_mae.append(mae)

print("MAE (Trained cGAN): {:.4f}".format(np.mean(g_cgan_avg_mae)))

NameError: ignored

The next piece of code will show coloured examples for both networks, so you can check them visually and discuss which model is better. First, we need to create an iterator object to go through the test dataset:

In [ ]:
iterator = iter(dataset_loader.load_batch(1, is_training=False))

Run multiple examples so that you have a clear idea of how both methods differ.

In [3]:
# Load test example
[imgs_A, imgs_B] = next(iterator)

# Generate predictions for both models
fake_A_cGAN = generator_cGAN.predict(imgs_B)
fake_A_MAE = generator_mae.predict(imgs_B)

# Plot all images
showColored_two_models_Ims(imgs_B, fake_A_MAE, fake_A_cGAN, imgs_A)

NameError: ignored

We showed that both models obtain a similar MAE value. If we would only take 
into account the quantitative metric, as done in many scientific articles, we would say that the MAE model is better. However, in addition to the quantitative results, we need to analyse visually the results produced by the two networks to declare which is the best model.

**Report**


*   Run the previous code to analyse several coloured images for both models. Based on previous results and linked to GAN theory, discuss from the numerical and visual perspective if both models are similar, or whether there is a better one. You can provide in the report visual examples together with their MAE values to support your arguments. The figure of this task can be included in the Appendix. Discussion still needs to go into the main text.